In [2]:
!git clone https://github.com/zoubohao/DenoisingDiffusionProbabilityModel-ddpm-

Cloning into 'DenoisingDiffusionProbabilityModel-ddpm-'...
remote: Enumerating objects: 144, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 144 (delta 50), reused 36 (delta 36), pack-reused 86 (from 1)
Receiving objects: 100% (144/144), 1.44 MiB | 8.89 MiB/s, done.
Resolving deltas: 100% (76/76), done.


In [13]:
cd DenoisingDiffusionProbabilityModel-ddpm-


/users/PFS0270/nikhilchatta/DenoisingDiffusionProbabilityModel-ddpm-


In [14]:
pip install -r requirements.txt


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install torch torchvision matplotlib tqdm


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [16]:
from torchvision.datasets import CIFAR10


In [53]:
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])

train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)


Files already downloaded and verified


In [54]:
!python Main.py


Using device: cpu
Training on device: cpu
Files already downloaded and verified
Epoch [1/5], Loss: 2.4760
Epoch [2/5], Loss: 0.2712
Epoch [3/5], Loss: 0.2398
Epoch [4/5], Loss: 0.1480
Epoch [5/5], Loss: 0.1736
Model saved to: ./Checkpoints/final_model.pt


In [57]:
pip install lpips


Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-2.0.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
Using cached numpy-2.0.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (19.5 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Not uninstalling numpy at /users/PFS0270/nikhilchatta/.local/lib/python3.9/site-packages, outside environment /apps/project/ondemand/app_jupyter/4.1.5
    Can't uninstall 'numpy'. No files were found to uninstall.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [60]:
# Cell 1: Shared Setup
import torch
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import torchvision.utils as vutils
import os

# Load Model
from Diffusion.Unet import UNet

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet()
model.load_state_dict(torch.load('./Checkpoints/final_model.pt', map_location=device))
model.to(device)
model.eval()

# Load Data
transform = transforms.ToTensor()
batch_size = 64
real_dataset = CIFAR10(root="./CIFAR10", train=True, download=True, transform=transform)
real_loader = DataLoader(real_dataset, batch_size=batch_size, shuffle=True)
real_imgs = next(iter(real_loader))[0].to(device)

# Generate Fake Images
with torch.no_grad():
    fake_imgs = model(real_imgs)

# Optionally clamp for image-based metrics
fake_imgs_clamped = torch.clamp((fake_imgs + 1) / 2, 0, 1)
real_imgs_clamped = torch.clamp((real_imgs + 1) / 2, 0, 1)


/tmp/slurmtmp.724782/ipykernel_1879658/572642896.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('./Checkpoints/final_model.pt', map_lo

Files already downloaded and verified


In [66]:
from piq import ssim  # use ssim instead of multi_scale_ssim for 32x32 images

ssim_scores = []

for _ in range(20):
    real_imgs = next(iter(real_loader))[0].to(device)
    with torch.no_grad():
        fake_imgs = model(real_imgs)

    fake_imgs_clamped = torch.clamp((fake_imgs + 1) / 2, 0, 1)
    real_imgs_clamped = torch.clamp((real_imgs + 1) / 2, 0, 1)

    score = ssim(fake_imgs_clamped, real_imgs_clamped, data_range=1.0).mean().item()
    ssim_scores.append(score)

avg_ssim = sum(ssim_scores) / len(ssim_scores)
print("Average SSIM over 20 rounds:", round(avg_ssim, 4))


Average SSIM over 20 rounds: 0.9988


In [62]:
!pip install piq


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [67]:
def calculate_psnr(img1, img2):
    mse = torch.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    psnr = 20 * torch.log10(1.0 / torch.sqrt(mse))
    return psnr.item()

psnr_scores = []

for _ in range(20):
    real_imgs = next(iter(real_loader))[0].to(device)
    with torch.no_grad():
        fake_imgs = model(real_imgs)

    score = calculate_psnr(fake_imgs, real_imgs)
    psnr_scores.append(score)

avg_psnr = sum(psnr_scores) / len(psnr_scores)
print("Average PSNR over 20 rounds:", round(avg_psnr, 4))


Average PSNR over 20 rounds: 39.0722


In [79]:
# Perceptual Similarity using VGG features (alternative to LPIPS)
import torch.nn.functional as F
from torchvision.models import vgg16
from torchvision import models

# Extract intermediate features from VGG
class VGGPerceptual(torch.nn.Module):
    def __init__(self):
        super(VGGPerceptual, self).__init__()
        vgg = vgg16(pretrained=True).features[:16]  # up to relu_3_3
        self.vgg = vgg.eval().to(device)
        for param in self.vgg.parameters():
            param.requires_grad = False

    def forward(self, x):
        return self.vgg(x)

vgg_perceptual = VGGPerceptual()

def perceptual_distance(x, y):
    fx = vgg_perceptual(x)
    fy = vgg_perceptual(y)
    return F.l1_loss(fx, fy).item()

perceptual_scores = []

for _ in range(20):
    real_imgs = next(iter(real_loader))[0].to(device)
    with torch.no_grad():
        fake_imgs = model(real_imgs)

    # Resize to 224x224 for VGG input
    real_resized = F.interpolate(real_imgs, size=(224, 224), mode='bilinear')
    fake_resized = F.interpolate(fake_imgs, size=(224, 224), mode='bilinear')

    score = perceptual_distance(real_resized, fake_resized)
    perceptual_scores.append(score)

avg_perceptual = sum(perceptual_scores) / len(perceptual_scores)
print("Average Perceptual Similarity (VGG-based) over 20 rounds:", round(avg_perceptual, 4))


/users/PFS0270/nikhilchatta/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/users/PFS0270/nikhilchatta/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /users/PFS0270/nikhilchatta/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:01<00:00, 432MB/s] 


Average Perceptual Similarity (VGG-based) over 20 rounds: 0.0234


In [32]:
# Create folders
import os
import torchvision.utils as vutils

os.makedirs("./Exported_FID/Real", exist_ok=True)
os.makedirs("./Exported_FID/Fake", exist_ok=True)

# Generate and export images
for i in range(20):  # 20 rounds/batches
    real_imgs = next(iter(real_loader))[0].to(device)
    with torch.no_grad():
        fake_imgs = model(real_imgs)

    # Clamp to [0, 1] for FID
    real_clamped = torch.clamp((real_imgs + 1) / 2, 0, 1)
    fake_clamped = torch.clamp((fake_imgs + 1) / 2, 0, 1)

    for j in range(real_clamped.shape[0]):
        vutils.save_image(real_clamped[j], f"./Exported_FID/Real/real_{i}_{j}.png")
        vutils.save_image(fake_clamped[j], f"./Exported_FID/Fake/fake_{i}_{j}.png")

print("Exported 20 batches of real/fake images for FID computation.")


NameError: name 'real_loader' is not defined

In [84]:
!pip install pytorch-fid


Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-2.0.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
Using cached numpy-2.0.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (19.5 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Not uninstalling numpy at /users/PFS0270/nikhilchatta/.local/lib/python3.9/site-packages, outside environment /apps/project/ondemand/app_jupyter/4.1.5
    Can't uninstall 'numpy'. No files were found to uninstall.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [86]:
import os
import torchvision.utils as vutils

os.makedirs("./Exported_FID/Real", exist_ok=True)
os.makedirs("./Exported_FID/Fake", exist_ok=True)

for i in range(20):
    real_imgs = next(iter(real_loader))[0].to(device)
    with torch.no_grad():
        fake_imgs = model(real_imgs)

    real_clamped = torch.clamp((real_imgs + 1) / 2, 0, 1)
    fake_clamped = torch.clamp((fake_imgs + 1) / 2, 0, 1)

    for j in range(real_clamped.shape[0]):
        vutils.save_image(real_clamped[j], f"./Exported_FID/Real/real_{i}_{j}.png")
        vutils.save_image(fake_clamped[j], f"./Exported_FID/Fake/fake_{i}_{j}.png")

print("Exported all images.")


Exported all images.


In [87]:
import os
os.listdir(".")


['.git',
 '.gitignore',
 'CIFAR10',
 'Checkpoints',
 'CheckpointsCondition',
 'Diffusion',
 'DiffusionFreeGuidence',
 'LICENSE',
 'Main.py',
 'MainCondition.py',
 'README.md',
 'SampledImgs',
 'Scheduler.py',
 'data',
 '.ipynb_checkpoints',
 '__pycache__',
 'Untitled.ipynb',
 'Untitled1.ipynb',
 'Exported_FID']

In [88]:
import os

# Print full absolute path
export_folder = os.path.abspath("Exported_FID")
print("Full path to Exported_FID folder:", export_folder)

# List contents to make sure it’s accessible
print("Contents of Exported_FID folder:")
print(os.listdir(export_folder))


Full path to Exported_FID folder: /users/PFS0270/nikhilchatta/DenoisingDiffusionProbabilityModel-ddpm-/Exported_FID
Contents of Exported_FID folder:
['Real', 'Fake', '.ipynb_checkpoints']


In [27]:
import shutil

# Zip Exported_FID folder inside current directory
shutil.make_archive('Exported_FID', 'zip', './Exported_FID')


KeyboardInterrupt: 

In [1]:
ls


data/                                      final.ipynb  status/
DenoisingClean/                            my-project/
DenoisingDiffusionProbabilityModel-ddpm-/  ondemand/
